<a href="https://colab.research.google.com/github/mesleki2017/plc-laboratuvar/blob/master/ML_yasin_lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd 
df=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/yasin-lab-veri/5-cycle-calisma-akim.csv',sep=',')
df.head()

,SeqNo,Date,UTC Time,sicaklik,basinc,titresim,araba-geride,araba-ileride,cene-acik,cene-kapali,devirme-yukarida,devirme-asagida,kaldirici-asagida,kaldirci-yukarida,kaldiriciyukari,kadiriciasagi,arabaileri,arabageri,cenekapa,ceneac,devirmeyukari,devirmeasagi,akim
0,1,2021-08-31,15:34:01.426,312.0,1059.0,134.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.040000
1,2,2021-08-31,15:34:01.438,312.0,1059.0,134.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.041000
2,3,2021-08-31,15:34:01.454,312.0,1059.0,134.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.042001
3,4,2021-08-31,15:34:01.473,312.0,1059.0,134.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.043000
4,5,2021-08-31,15:34:01.487,312.0,1059.0,134.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.040000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8821 entries, 0 to 8820
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SeqNo              8821 non-null   object 
 1   Date               8820 non-null   object 
 2   UTC Time           8820 non-null   object 
 3   sicaklik           8820 non-null   float64
 4   basinc             8820 non-null   float64
 5   titresim           8820 non-null   float64
 6   araba-geride       8820 non-null   float64
 7   araba-ileride      8820 non-null   float64
 8   cene-acik          8820 non-null   float64
 9   cene-kapali        8820 non-null   float64
 10  devirme-yukarida   8820 non-null   float64
 11  devirme-asagida    8820 non-null   float64
 12  kaldirici-asagida  8820 non-null   float64
 13  kaldirci-yukarida  8820 non-null   float64
 14  kaldiriciyukari    8820 non-null   float64
 15  kadiriciasagi      8820 non-null   float64
 16  arabaileri         8820 

# SeqNo kolunundaki 1 fazla satırın silinmesi
bu satır en sondaki END satırı

In [ ]:
print(df.isna().sum())

SeqNo                0
Date                 1
UTC Time             1
sicaklik             1
basinc               1
titresim             1
araba-geride         1
araba-ileride        1
cene-acik            1
cene-kapali          1
devirme-yukarida     1
devirme-asagida      1
kaldirici-asagida    1
kaldirci-yukarida    1
kaldiriciyukari      1
kadiriciasagi        1
arabaileri           1
arabageri            1
cenekapa             1
ceneac               1
devirmeyukari        1
devirmeasagi         1
akim                 1
dtype: int64


In [ ]:
df["SeqNo"]

0                 1
1                 2
2                 3
3                 4
4                 5
           ...     
8816           8817
8817           8818
8818           8819
8819           8820
8820          //END
Name: SeqNo, Length: 8821, dtype: object

In [ ]:
df = df.drop(8820)
print(df.isna().sum())

SeqNo                0
Date                 0
UTC Time             0
sicaklik             0
basinc               0
titresim             0
araba-geride         0
araba-ileride        0
cene-acik            0
cene-kapali          0
devirme-yukarida     0
devirme-asagida      0
kaldirici-asagida    0
kaldirci-yukarida    0
kaldiriciyukari      0
kadiriciasagi        0
arabaileri           0
arabageri            0
cenekapa             0
ceneac               0
devirmeyukari        0
devirmeasagi         0
akim                 0
dtype: int64


# temel DF nin DF1 olarak kopyalanması

In [ ]:
df1=df.copy()

# PLC data scala işlemi , bu işlemler PLC yapısı ile alakalı

In [ ]:
sabit1=10*25/65366
df1["titresim"]=df["titresim"]*sabit1
#print(df1["titresim"].describe())

sabit2=1/10
df1["sicaklik"]=df["sicaklik"]*sabit2
#print(df1["sicaklik"].describe())

sabit3=1/100
df1["basinc"]=df["basinc"]*sabit3
#print(df1["basinc"].describe())


# Date kolonu sadece yıl ay gün bilgisi içerdiği için bu kolonun silinmesi

In [ ]:
df1=df1.drop(["Date"],axis=1)

In [ ]:
df1.head()

,SeqNo,UTC Time,sicaklik,basinc,titresim,araba-geride,araba-ileride,cene-acik,cene-kapali,devirme-yukarida,devirme-asagida,kaldirici-asagida,kaldirci-yukarida,kaldiriciyukari,kadiriciasagi,arabaileri,arabageri,cenekapa,ceneac,devirmeyukari,devirmeasagi,akim
0,1,15:34:01.426,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.040000
1,2,15:34:01.438,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.041000
2,3,15:34:01.454,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.042001
3,4,15:34:01.473,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.043000
4,5,15:34:01.487,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.040000


# Hareket aralıkları için zaman aralık süresi eklemek, milli saniye olarak 

In [ ]:
df_zaman=df1.drop(["SeqNo","UTC Time","sicaklik","basinc","titresim","akim"],axis=1)

In [ ]:
aaa=df_zaman.diff()

In [ ]:
dfr=(aaa == 0.0)

In [ ]:
bbb=dfr.all(axis=1)

In [ ]:
df_zaman["degisim"]=bbb

In [ ]:
liste7=df_zaman.loc[df_zaman['degisim'] == False].index.tolist()
print("degisimlerin oldugu satir no listesi \n",liste7)

degisimlerin oldugu satir no listesi 
 [0, 128, 144, 267, 274, 702, 708, 736, 741, 1077, 1081, 1551, 1569, 1615, 1624, 2332, 2348, 2450, 2451, 2496, 2509, 2597, 2605, 2945, 2951, 2972, 2978, 3248, 3253, 3632, 3633, 3651, 3681, 3682, 3688, 4279, 4292, 4375, 4427, 4439, 4517, 4522, 4802, 4805, 4823, 4826, 5046, 5049, 5366, 5380, 5412, 5417, 5917, 5929, 6004, 6046, 6056, 6125, 6129, 6372, 6374, 6391, 6394, 6585, 6588, 6864, 6877, 6905, 6910, 7366, 7367, 7378, 7446, 7477, 7486, 7545, 7548, 7780, 7783, 7793, 7797, 7975, 7978, 8231, 8241, 8269, 8273, 8691, 8701, 8763]


In [ ]:
df_zaman_ekle=df1.copy()
df_zaman_ekle["hareket-zaman-aralik"]=0

In [ ]:
from datetime import datetime 

for i,zz in enumerate(liste7):
  ilk=df_zaman_ekle.loc[liste7[i]]["UTC Time"]
  d5=datetime.strptime(ilk,"%H:%M:%S.%f")
  if i+1 != len(liste7):
    son=df_zaman_ekle.loc[liste7[i+1]]["UTC Time"]
    d6=datetime.strptime(son,"%H:%M:%S.%f")
    df_zaman_ekle.at[liste7[i], "hareket-zaman-aralik"] = (d6.timestamp()-d5.timestamp())*1000


In [ ]:
df_zaman_ekle

,SeqNo,UTC Time,sicaklik,basinc,titresim,araba-geride,araba-ileride,cene-acik,cene-kapali,devirme-yukarida,devirme-asagida,kaldirici-asagida,kaldirci-yukarida,kaldiriciyukari,kadiriciasagi,arabaileri,arabageri,cenekapa,ceneac,devirmeyukari,devirmeasagi,akim,hareket-zaman-aralik
0,1,15:34:01.426,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.040000,2532
1,2,15:34:01.438,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.041000,0
2,3,15:34:01.454,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.042001,0
3,4,15:34:01.473,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.043000,0
4,5,15:34:01.487,31.2,10.59,0.512499,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.040000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8815,8816,15:38:41.082,31.6,10.36,0.596640,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.006001,0
8816,8817,15:38:41.119,31.6,10.36,0.596640,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.006001,0
8817,8818,15:38:41.159,31.6,10.53,0.585167,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.007000,0
8818,8819,15:38:41.196,31.6,10.53,0.585167,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.000000,0


# Train Test SET

In [ ]:
train_set, test_set = train_test_split(df1, test_size=0.2, random_state=42)

# Feauters and Labels

In [ ]:
df_feature=train_set[["araba-geride","araba-ileride","cene-acik","cene-kapali","devirme-yukarida","devirme-asagida","kaldirici-asagida","kaldirci-yukarida"]]
df_feature.head()

,araba-geride,araba-ileride,cene-acik,cene-kapali,devirme-yukarida,devirme-asagida,kaldirici-asagida,kaldirci-yukarida
3082,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
449,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
8125,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
8395,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7772,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
df_labels=train_set.drop(["SeqNo","UTC Time","araba-geride","araba-ileride","cene-acik","cene-kapali","devirme-yukarida","devirme-asagida","kaldirici-asagida","kaldirci-yukarida","akim","sicaklik","basinc","titresim"], axis=1)
df_labels.head()

,kaldiriciyukari,kadiriciasagi,arabaileri,arabageri,cenekapa,ceneac,devirmeyukari,devirmeasagi
3082,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
449,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8125,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8395,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7772,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Numerical and Categorical attribs

In [ ]:
num_attribs=["araba-geride","araba-ileride","cene-acik","cene-kapali","devirme-yukarida","devirme-asagida","kaldirici-asagida","kaldirci-yukarida"]
# cetegorical attribs YOK

# Pipline

In [ ]:
num_pipeline = Pipeline([

        ('std_scaler', StandardScaler()),
    ])
# eksik data olmadigi icin imputer kullanmadim

In [ ]:
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
    ])
# categorical data olmadigi icin onhotencoder kullanmadim

# Features Pipline preprocessed

In [ ]:
df_feature_preprocessed= full_pipeline.fit_transform(df_feature)
print (df_feature_preprocessed[10])

[-0.94483007 -0.1747121   0.75991104 -0.71207322  0.40184758 -0.22395596
  0.68601221 -0.12349817]


# some data

In [ ]:
some_data = df_feature.iloc[2000:2100]
some_labels = df_labels.iloc[2000:2100]
some_labels

,kaldiriciyukari,kadiriciasagi,arabaileri,arabageri,cenekapa,ceneac,devirmeyukari,devirmeasagi
8364,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8609,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8170,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6716,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1366,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
3805,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7214,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2654,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4112,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## some data pipline preprocessed

In [ ]:
some_data_prepared = full_pipeline.transform(some_data)
some_data_prepared[0]

array([-0.94483007, -0.1747121 ,  0.75991104, -0.71207322,  0.40184758,
       -0.22395596,  0.68601221, -0.12349817])

# DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(df_feature_preprocessed, df_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

# Some Data Prediction

In [ ]:
index5=90
df1_predictions = tree_reg.predict(some_data_prepared)
print("Predictions:", df1_predictions[index5])
print("\n","------------------------------------------------","\n")
print("Labels: \n", some_labels.iloc[index5])

Predictions: [1. 0. 0. 0. 0. 0. 0. 0.]

 ------------------------------------------------ 

Labels: 
 kaldiriciyukari    1.0
kadiriciasagi      0.0
arabaileri         0.0
arabageri          0.0
cenekapa           0.0
ceneac             0.0
devirmeyukari      0.0
devirmeasagi       0.0
Name: 7703, dtype: float64


# DecisionTreeRegressor Root Mean Square Error

## DecisionTreeRegressor full Train data prediction

In [ ]:
df1_predictions = tree_reg.predict(df_feature_preprocessed)

## Root Mean Square Error

In [ ]:
tree_mse = mean_squared_error(df_labels, df1_predictions)

In [ ]:
tree_rmse = np.sqrt(tree_mse)
print("DecisionTreeRegressor_rmse : ",tree_rmse)

DecisionTreeRegressor_rmse :  0.08478792350235552
